In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
import holoviews as hv
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
df_market_data_scaled=StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h","price_change_percentage_7d",
                                                                     "price_change_percentage_14d", "price_change_percentage_30d",
                                                                     "price_change_percentage_60d", "price_change_percentage_200d",
                                                                     "price_change_percentage_1y"]]
                                                    )
                                                                     
                                                            

In [6]:
# Create a DataFrame with the scaled data
df_market_data_scaled=pd.DataFrame(df_market_data_scaled,
                                   columns=["price_change_percentage_24h","price_change_percentage_7d",
                                                                     "price_change_percentage_14d", "price_change_percentage_30d",
                                                                     "price_change_percentage_60d", "price_change_percentage_200d",
                                                                     "price_change_percentage_1y"]
                                  )

# Copy the crypto names from the original DataFrame
df_market_data_scaled["coin_id"]=df_market_data.index

# Set the coin_id column as index
df_market_data_scaled=df_market_data_scaled.set_index("coin_id")

# Display the scaled DataFrame
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [7]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [8]:
# Create an empty list to store the inertia values
inertia=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model=KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)


In [9]:
print(inertia)

[287.0, 212.1233420748626, 165.1367523344681, 79.02243535120975, 66.41305075939437, 61.634088110682214, 46.074467441030215, 43.39269411734827, 32.913222659715245, 30.469111293270352]


In [10]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [11]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x = "k",
    y = "inertia",
    title = "Elbow Curve",
    xticks = k
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** From the elbow curve, it looks like the best k value is 4.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [12]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [13]:
# Fit the K-Means model using the scaled DataFrame
model.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=1)

In [14]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
k_lower = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(k_lower)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [15]:
# Create a copy of the scaled DataFrame
df_market_data_predictions = df_market_data_scaled.copy()

In [16]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_market_data_predictions["cluster_group"]=k_lower

# Display the copy of the scaled DataFrame
df_market_data_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,cluster_group
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [17]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_market_data_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by = "cluster_group",
    hover_cols=["coin_id"]
)

:NdOverlay   [cluster_group]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [18]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA (n_components=3)

In [19]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
market_data_pca = pca.fit_transform(df_market_data_scaled)

# View the scaled PCA data
market_data_pca[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [20]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** About 89% of the total variance is condensed into the 3 PCA variables.

In [21]:
# Create a new DataFrame with the PCA data.
market_data_pca_df = pd.DataFrame(
    market_data_pca,
    columns=["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original scaled DataFrame
market_data_pca_df["coin_id"] = df_market_data_scaled.index

# Set the coin_id column as index
market_data_pca_df = market_data_pca_df.set_index("coin_id")

# Display the scaled PCA DataFrame
market_data_pca_df.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [22]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [23]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(market_data_pca_df)
    inertia.append(k_model.inertia_)


In [24]:
print(inertia)

[256.8740855678923, 182.33953007775636, 135.442407624542, 49.665496651797326, 38.67258217591795, 34.50757653758153, 23.728364781274763, 18.574523440183192, 15.572672210433335, 11.407666572096915]


In [25]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k":k,"inertia":inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

In [26]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow_pca.hvplot.line(
    x = "k",
    y = "inertia",
    title = "Elbow Curve w/ PCA",
    xticks = k
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** Looking at the elbow curve, the best value seems to be 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** k value is the same as the value found using the original data.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [27]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [28]:
# Fit the K-Means model using the PCA data
model.fit(market_data_pca_df)

KMeans(n_clusters=4, random_state=1)

In [29]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
k_4 = model.predict(market_data_pca_df)

# Print the resulting array of cluster values.
print(k_4)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [30]:
# Create a copy of the scaled PCA DataFrame
market_data_pca_predictions = market_data_pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
market_data_pca_predictions["cluster_pca"]=k_4

# Display the copy of the scaled PCA DataFrame
market_data_pca_predictions.head()

,PC1,PC2,PC3,cluster_pca
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [31]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
market_data_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by = "cluster_pca",
    hover_cols=["coin_id"]
)
    
    

:NdOverlay   [cluster_pca]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [32]:
# Composite plot to contrast the Elbow curves
# Create the first Elbow curve plot (df_elbow)
elbow_plot = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve (Original Scaled Data)", 
    xticks=k
)

# Create the second Elbow curve plot (df_elbow_pca)
elbow_plot_pca = df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve (PCA Data)", 
    xticks=k
)

# Combine both plots side by side using the '+' operator
composite_plot = elbow_plot + elbow_plot_pca

# Display the combined plot
composite_plot


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [33]:
# Composite plot to contrast the clusters
# Create the first scatter plot (df_market_data_predictions)
scatter_plot = df_market_data_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster_group",
    hover_cols=["coin_id"],
    title="Clusters: Original Scaled Data"
)

# Create the second scatter plot (market_data_pca_predictions)
scatter_plot_pca = market_data_pca_predictions.hvplot.scatter(
    x="PC1",  # Use PCA components for the axes
    y="PC2",
    by="cluster_pca",  # Cluster predictions based on PCA
    hover_cols=["coin_id"],
    title="Clusters: PCA Reduced Data"
)

# Combine both scatter plots side by side using the '+' operator
composite_scatter_plot = scatter_plot + scatter_plot_pca

# Display the combined scatter plot
composite_scatter_plot


:Layout
   .NdOverlay.I  :NdOverlay   [cluster_group]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [cluster_pca]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** On the PCA-reduced data, clusters appear tighter. PCA effectively captured the most significant variance in the data while filtering out noise and irrelevant features. With fewer dimensions, K-Means formed more compact and well-separated clusters. Although some outliers remain in the graph, the overall clustering is noticeably improved compared to the original data before PCA optimization.

    Examining the inertia values further supports this improvement. Before PCA, inertia starts at 287 and decreases as the number of clusters increases. After PCA, the initial inertia is already lower at 256.87 and continues to drop. This indicates that PCA successfully reduced noise and redundant dimensions, leading to more compact clusters. The reduction in inertia is particularly significant for lower k-values, suggesting that PCA enhanced the natural grouping of the data and improved cluster definition.